#### Advanced Statistics for Data Science (Spring 2022)
# Home Assignment 4
#### Topics:
- Multiple Comparison
- Simple Regression
- Multiple Regression

#### Due: 06/06/2023 by 18:30 
(Note: no class on this day!)

#### Instructions:
- Write your name, Student ID, and date in the cell below. 
- Submit a copy of this notebook with code filled in the relevant places as the solution of coding exercises.
- For theoretic exercises, you can either write your solution in the notebook using $\LaTeX$ or submit additional notes.

<hr>
<hr>

**Name**: Roni Ben Dom

**Student ID**: 207576463

**Date**: 01/06/2023

$
\newcommand{\Id}{{\mathbf{I}}}  
\newcommand{\SSE}{\mathsf{SSE}}
\newcommand{\SSR}{\mathsf{SSR}}
\newcommand{\MSE}{\mathsf{MSE}}
\newcommand{\simiid}{\overset{iid}{\sim}}
\newcommand{\ex}{\mathbb E}
\newcommand{\var}{\mathrm{Var}}
\newcommand{\Cov}[2]{{\mathrm{Cov}  \left(#1, #2 \right)}}
\newcommand{\one}[1]{\mathbf 1 {\left\{#1\right\}}}
\newcommand{\SE}[1]{\mathrm{SE} \left[#1\right]}
\newcommand{\reals}{\mathbb R}
\newcommand{\Ncal}{\mathcal N}
\newcommand{\abs}[1]{\ensuremath{\left\vert#1\right\vert}}
\newcommand{\rank}{\operatorname{rank}}
\newcommand{\tr}{\operatorname{Tr}}
\newcommand{\diag}{\operatorname{diag}}
\newcommand{\sign}{\operatorname{sign}}
$


<hr>
<hr>

## Problem 1 (Exact size of Bonferroni's test)
Suppose that we run multiple tests with independent data and obtain P-values $p_1,\ldots,p_n$. We wish to test the null hypothesis:
$$
H_0\,:\,\text{All tests are null}
$$
at the level $\alpha$ (e.g., $\alpha=0.05$). In class, we introduced Bonferroni's procedure which is equivalent to: Reject $H_0$ if $\min p_i \leq \alpha/n$. 
1. Show that the  size of the test in Bonferroni's procedure is at most $\alpha$, regardless if the hypotheses are independent or not. 
2. Assuming that the hypotheses are independent, find the exact size of the test in Bonferroni's procedure. 
3. For $\alpha=0.05$, evaluate the difference between $\alpha$ and the exact test's size for $n=2,...,50$. Discuss what you see.



1. 1. Bonferroni's test staits that if we have total of n t-tests we conduct each test at the level of $\frac{\alpha}{n}$ instead of $\alpha$, meaning:
$$\text{Pr}\left(\text{reject}\; H_{0}\right|H_{0}\; \text{is true}) = \text{Pr}\left(min\left(p_{i}\right)\leq\frac{\alpha}{n}\right) = \\
\text{Pr}\left(p_{1} \leq \frac{\alpha}{n}\cup p_{2} \leq \frac{\alpha}{n}\cup\dots\cup p_{n} \leq \frac{\alpha}{n}\right) \underbrace{\leq}_{\text{union}} \sum_{i=1}^n \text{Pr}\left(p_{i}\leq\frac{\alpha}{n}\right) = \sum_{i=1}^n \frac{\alpha}{n} = n\cdot\frac{\alpha}{n} = \alpha$$<br>
<br>
  2. $$\text{Pr}\left(\text{reject}\; H_{0}\right|H_{0}\; \text{is true}) = \text{Pr}\left(min\left(p_{i}\right)\leq\frac{\alpha}{n}\right) = \\
\text{Pr}\left(p_{1} \leq \frac{\alpha}{n}\cup p_{2} \leq \frac{\alpha}{n}\cup\dots\cup p_{n} \leq \frac{\alpha}{n}\right) =\\ 1 - \text{Pr}\left(p_{1} > \frac{\alpha}{n}\cap p_{2} > \frac{\alpha}{n}\cap\dots\cap p_{n} > \frac{\alpha}{n}\right) \underbrace{=}_{\text{the hypotheses are independant}}\\
1 - \prod_{i=1}^n\text{Pr}\left(p_{i}>\frac{\alpha}{n}\right) = 1 - \prod_{i=1}^n\left(1 - \text{Pr}\left(p_{i}\leq\frac{\alpha}{n}\right)\right) = 1 - \prod_{i=1}^n\left(1 - \frac{\alpha}{n}\right) = 1 - \left(1 - \frac{\alpha}{n}\right)^n$$
<br>
  3.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

alpha = 0.05
n_values = np.arange(2, 51)
exact_size = [1-(1-(alpha/n))**n for n in n_values]

fig = go.Figure()

fig.add_trace(go.Scatter(
        x=n_values,
        y=exact_size,
        mode='lines+markers',
        name='Exact size',
        line=dict(color='#ff7f0e'),
        showlegend=True
    ))

fig.add_trace(go.Scatter(
        x=n_values,
        y=np.full(len(n_values), alpha),
        mode='lines',
        name=r'$\alpha$',
        line=dict(color='#1f77b4'),
        showlegend=True
    ))

fig.update_layout(
        height=500, width=1200,
        title=dict(text=r"$\text{Exact size vs. }\alpha$", x=0.5),
        xaxis_title="n",
        yaxis_title="exact size",
        template='plotly_white',
        legend=dict(orientation='h', x=.5, y=1, xanchor='center', yanchor='bottom')
    )

fig.show()
    

  As we can see from the plot above, the exact value decreases when n increases, meaning he p-value threshold for rejecting the Null hypotheses decreases. This can cause to fail to reject hypothesis that might should be rejected and can be rejected over bigger population.

## Problem 2 (Prediction in Simple Regression)
Consider the linear model:
$$
    y_i = \beta_0 + \beta_1 x_i + \epsilon_i,\qquad \epsilon_i \simiid \Ncal(0,\sigma^2)
$$
$$
    Z = \begin{pmatrix}
    1 & x_1 \\
    \vdots & \vdots \\
    1 & x_n
    \end{pmatrix},\quad \beta=\begin{pmatrix}
    \beta_0 \\
    \beta_1
    \end{pmatrix},\qquad \hat{\beta}=(Z^\top Z)^{-1} Z^\top y
$$
Suppose we get a new data point $x_{n+1}$ and want to predict $y_{n+1}$. We want an interval in which this prediction will likely to land. In class, we used that 
$$
\var[\hat{\beta}_0 + \hat{\beta}_1 x] = \sigma^2 \left( \frac{1}{n} + \frac{(x-\bar{x})^2}{S_{XX }}\right)
$$
to obtain a confidence interval for $\beta_0 + \beta_1 x$, and a confidence band for all $x \in \reals$. In this question, you will use a similar reasoning to get a confidence interval (and bands) for $y_{n+1}$.
1. Find the variance of $y_{n+1} - (\hat{\beta}_0 + \hat{\beta}_1 x_{n+1})$ in terms of $\sigma^2$ and $x_1,\ldots,x_n$ and $x_{n+1}$ (you can use $\bar{x}$ and $S_{XX}$ or any other well-defined function of $x_1,\ldots,x_n$). Explain intuitively why it makes sense that this variance is larger than the variance of  $\hat{\beta}_0 + \hat{\beta}_1 x_{n+1}$.
2. Find a $1-\alpha$ confidence interval for $y_{n+1}$. Is this interval wider or narrower than that of $\hat{\beta}_0 + \hat{\beta}_1 x_{n+1}$? For what value of $x_{n+1}$ this interval is the narrowest?
3. Suppose that we take the average of two responses $y$'s at the same $x_{n+1}$, say 
$$
y_{n+1} =  \frac{y_{n+1}^{(1)} + y_{n+1}^{(2)}}{2}, 
$$
where
$$
y_{n+1}^{(1)} = \beta_0 + \beta_1 x_{n+1} + \epsilon_{n+1}^{(1)}
$$
and 
$$
y_{n+1}^{(2)} = \beta_0 + \beta_1 x_{n+1} + \epsilon_{n+1}^{(2)},
$$
where $\epsilon_{n+1}^{(1)}$ and $\epsilon_{n+1}^{(2)}$ are independent. Find a confidence interval for $y_{n+1}$. Is it wider or narrower than the interval in (2) ?

Note: The confidence interval you derived in 2 is somewhat risky to use because it makes the strong assumption that $\epsilon_{n+1}$ is normal. This is compared to, say, confidence intervals for $\hat{\beta}_0$ and $\hat{\beta}_1$ which rely on averages over all observations $y_1,\ldots,y_n$ so we can use the Central Limit Theorem to argue for normality. Things get better both in terms of variance and normality when you can take multiple measurements at the same $x_{n+1}$ and average these measurements.


2. 1. $\displaystyle{\operatorname{Var}\left[y_{n+1} - \left(\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right)\right] = \operatorname{Var}\left[\beta_0 + \beta_1 x_{n+1} + \epsilon_{n+1} - \left(\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right)\right] \underbrace{=}_{\text{from the lecture}}\\ \operatorname{Var}\left[\epsilon_{n+1} - \left(\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right)\right] = \operatorname{Var}\left[\epsilon_{n+1}\right] +\operatorname{Var}\left[\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right]-2\operatorname{Cov}\left(\epsilon_{n+1}, \hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right) = \\
\sigma^2 +\sigma^2 \left( \frac{1}{n} + \frac{(x-\bar{x})^2}{S_{XX }}\right)-2\operatorname{Cov}\left(\epsilon_{n+1}, \hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right) =\\ \sigma^2 +\sigma^2 \left( \frac{1}{n} + \frac{(x-\bar{x})^2}{S_{XX }}\right) = \sigma^2 \cdot\left(1 + \frac{1}{n} + \frac{(x-\bar{x})^2}{S_{XX }}\right) > \var[\hat{\beta}_0 + \hat{\beta}_1 x]}$
<br>
<br>
  The variance is bigger because of the noise addded by $\epsilon$ (independant of $\hat{\beta}_{0}, \hat{\beta}_{1}$), which has a variance of $\sigma^2$ which is exactly the difference between $\operatorname{Var}\left[y_{n+1} - \left(\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right)\right]$ and $\var[\hat{\beta}_0 + \hat{\beta}_1 x]$
<br>
<br>
  2. Based on the proccess from the lecture:<br>
  $\displaystyle{\frac{y_{n+1}-\left(\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right)}{s\sqrt{1 + \frac{1}{n} + \frac{(x-\bar{x})^2}{S_{XX }}}}\sim t_{n-2}}\Rightarrow$ confidence interval for $y_{n+1}$ is: $\;\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1} \pm t_{n-2}^{1-\frac{\alpha}{2}}s\sqrt{1 + \frac{1}{n} + \frac{(x-\bar{x})^2}{S_{XX }}}$<br>
  This interval is wider than that of $\beta_{0}+\beta_{1}x_{n+1}$. <br>
  This interval is the narrowest for $x = \bar{x}$ cause than the term $\frac{(x-\bar{x})^2}{S_{XX }}$ is 0.
  <br>
  <br>
  3. $\displaystyle{\operatorname{Var}\left[y_{n+1} - \left(\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right)\right] = \operatorname{Var}\left[  \frac{y_{n+1}^{(1)} + y_{n+1}^{(2)}}{2} - \left(\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right)\right] = \operatorname{Var}\left[  \frac{  \epsilon_{n+1}^{(1)}
 + \epsilon_{n+1}^{(2)}}{2} - \left(\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right)\right] \underbrace{=}_{\text{from the lecture}} \operatorname{Var}\left(\frac{\epsilon_{n+1}^{(1)}}{2}\right)+\operatorname{Var}\left(\frac{\epsilon_{n+1}^{(2)}}{2}\right)+\operatorname{Var}\left(\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right) =\\ 0.25\operatorname{Var}\left(\epsilon_{n+1}^{(1)}\right)+0.25\operatorname{Var}\left(\epsilon_{n+1}^{(2)}\right)+\operatorname{Var}\left(\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1}\right) = \\ 0.25\cdot \sigma^2 + 0.25 \cdot \sigma^2 +\sigma^2 \left( \frac{1}{n} + \frac{(x-\bar{x})^2}{S_{XX }}\right) = \\ 0.5\cdot \sigma^2 +\sigma^2 \left( \frac{1}{n} + \frac{(x-\bar{x})^2}{S_{XX }}\right) = \sigma^2 \left(0.5 + \frac{1}{n} + \frac{(x-\bar{x})^2}{S_{XX }}\right)\\ \qquad\qquad\qquad\qquad\qquad\Downarrow}$<br>
 confidence interval for $y_{n+1}$ is: $\;\hat{\beta}_{0}+\hat{\beta}_{1}x_{n+1} \pm t_{n-2}^{1-\frac{\alpha}{2}}s\sqrt{0.5 + \frac{1}{n} + \frac{(x-\bar{x})^2}{S_{XX }}}$<br>
 This confidence interval is narrower than the one in (2), which makes sense bacause we take into acount more options, hence has more knowledge causing the confidence interval to be narrower.

## Problem 3 (Multiple Regression)

Use the house prices dataset from class. Load it using the function ``load_house_prices_data``. We will use ``LogSalePrice`` as the target variable (note that this is a transformed version of the original sale prince)

1. Use all variables in the data returned by ``load_house_prices_data``. Find at least one pair of competing predictors.
2. A construction company is trying to figure out how to design a new development for maximal profit. They propose that since ``SecondFlrSF`` (second floor square footage, proportional to second floor square meter) is correlated with ``LogSalePrice``, they should try and maximize second floor area in their designs. Does this conclusion make sense considering that ``SecondFlrSF`` is correlated with``TotRmsAbvGrd``? offer a procedure that checks the effect of ``SecondFlrSF`` on ``LogSalePrice`` and gives more information to make such decision.


In [2]:
def load_house_prices_data(path = "housing_prices.csv"):
    """
    Args:
    -----
    path:  path to csv file
    
    Load and clean house prices data:
        filters for numeric predictors only
        filters for small lots only
        renames varaibles so that all variable names begins with [a-z]
        applies a variance stabilizing transformation to SalePrice
        removes outliers
        
    """
    
    
    def detect_outliers(df, q=0.01):
        lower_outliers = df < df.quantile(q)
        upper_outliers = df > df.quantile(1-q)
        return lower_outliers | upper_outliers
    
    data_raw = pd.read_csv(path)
    data1 = data_raw[data_raw.LotArea < 15000]  # focus on small lots
    data1 = data1.select_dtypes('number').dropna()
    data1 = data1.rename( # stats model formula cannot have
        # covaraite names starting with non letter
        columns = {'1stFlrSF': 'FirstFlrSF',
                   '2ndFlrSF': 'SecondFlrSF'}) 
                                                              
    variables =[
        'SalePrice',
        'LotArea', 
        'YearBuilt',
         'YrSold', 'MoSold', 
         'Fireplaces', 
        'GarageCars', 'ScreenPorch', 
         'HalfBath', 'FullBath',
         'GrLivArea', 
         'BedroomAbvGr',
        'FirstFlrSF', 
        'SecondFlrSF',
        'TotRmsAbvGrd',
        'LowQualFinSF', 'TotalBsmtSF',
        'LotFrontage', 'WoodDeckSF',
         'OverallQual',
         'OverallCond'
    ]

    data1 = data1.filter(variables).dropna()
    data1['LogSalePrice'] = np.log(1 + data1['SalePrice'])
    data1 = data1.drop('SalePrice', axis=1)
    
    mask = detect_outliers(data1, .01).any(1)
    print(f"Masked a fraction of {mask.mean()} of the data due to outliers")
    return data1[~mask]


In [3]:
data = load_house_prices_data()
data.head()

Masked a fraction of 0.14971209213051823 of the data due to outliers


<ipython-input-2-1be3f286ff85>:53: FutureWarning:

In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.



,LotArea,YearBuilt,YrSold,MoSold,Fireplaces,GarageCars,ScreenPorch,HalfBath,FullBath,GrLivArea,...,FirstFlrSF,SecondFlrSF,TotRmsAbvGrd,LowQualFinSF,TotalBsmtSF,LotFrontage,WoodDeckSF,OverallQual,OverallCond,LogSalePrice
0,8450,2003,2008,2,0,2,0,1,2,1710,...,856,854,8,0,856,65.0,0,7,5,12.247699
1,9600,1976,2007,5,1,2,0,0,2,1262,...,1262,0,6,0,1262,80.0,298,6,8,12.109016
2,11250,2001,2008,9,1,2,0,1,2,1786,...,920,866,6,0,920,68.0,0,7,5,12.317171
3,9550,1915,2006,2,1,3,0,0,1,1717,...,961,756,7,0,756,60.0,0,7,5,11.849405
4,14260,2000,2008,12,1,3,0,1,2,2198,...,1145,1053,9,0,1145,84.0,192,8,5,12.429220


In [4]:
corr = data.corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='RdBu_r', axis=None, vmin=-1, vmax=1)
 .highlight_null(color='#f1f1f1')  # Color of NaNs: grey
 .format(precision=2))

,LotArea,YearBuilt,YrSold,MoSold,Fireplaces,GarageCars,ScreenPorch,HalfBath,FullBath,GrLivArea,BedroomAbvGr,FirstFlrSF,SecondFlrSF,TotRmsAbvGrd,LowQualFinSF,TotalBsmtSF,LotFrontage,WoodDeckSF,OverallQual,OverallCond,LogSalePrice
LotArea,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
YearBuilt,0.09,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
YrSold,-0.03,0.02,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
MoSold,0.03,0.03,-0.16,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Fireplaces,0.23,0.11,-0.03,0.06,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
GarageCars,0.24,0.59,-0.04,0.08,0.22,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
ScreenPorch,0.07,-0.07,0.04,-0.03,0.14,-0.01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
HalfBath,0.07,0.21,-0.00,-0.02,0.17,0.16,0.03,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
FullBath,0.10,0.59,0.01,0.09,0.23,0.54,-0.03,0.12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
GrLivArea,0.33,0.29,-0.02,0.11,0.45,0.48,0.07,0.44,0.61,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [5]:
import statsmodels.formula.api as smf
from scipy.stats import pearsonr

alpha = 0.05
target_col = 'LogSalePrice'
predictors_cols = [column for column in data.columns if column!='LogSalePrice']
full_model = smf.ols(f"{target_col} ~ {' + '.join(predictors_cols)} + 1", data).fit()

for col in predictors_cols:
  partial_predictor_cols = predictors_cols.copy()
  partial_predictor_cols.remove(col)

  partial_model = smf.ols(f"{target_col} ~ {' + '.join(partial_predictor_cols)} + 1", data).fit()
  # print(partial_model.pvalues)
  for col2 in partial_predictor_cols:
    if partial_model.pvalues[col2] < alpha and full_model.pvalues[col2] > alpha:
      print(f"'{col2}' and '{col}' are competing predictors")
      r, p = pearsonr(data[col], data[col2])
      print(f"The correlation coef p-value of '{col2}' and '{col}' is: {p}")


'LotFrontage' and 'LotArea' are competing predictors
The correlation coef p-value of 'LotFrontage' and 'LotArea' is: 2.689931068360056e-142
'FullBath' and 'YearBuilt' are competing predictors
The correlation coef p-value of 'FullBath' and 'YearBuilt' is: 1.2261714427859819e-85
'LotFrontage' and 'YearBuilt' are competing predictors
The correlation coef p-value of 'LotFrontage' and 'YearBuilt' is: 0.0005490222310297407
'LotFrontage' and 'MoSold' are competing predictors
The correlation coef p-value of 'LotFrontage' and 'MoSold' is: 0.028990828188352132
'LotFrontage' and 'Fireplaces' are competing predictors
The correlation coef p-value of 'LotFrontage' and 'Fireplaces' is: 1.9910187412820106e-08
'LotFrontage' and 'GarageCars' are competing predictors
The correlation coef p-value of 'LotFrontage' and 'GarageCars' is: 2.532145487909041e-16
'FirstFlrSF' and 'GrLivArea' are competing predictors
The correlation coef p-value of 'FirstFlrSF' and 'GrLivArea' is: 3.7114277075406452e-34
'SecondFlr

2. 

In [6]:
import statsmodels.api as sm

varX = 'SecondFlrSF'
varY = 'LogSalePrice'
varZ = 'TotRmsAbvGrd' # total rooms above ground level

model_SecondFlrSF = smf.ols(formula= f"{varX} ~ {varZ}", data=data).fit()
model_SalePrice = smf.ols(formula= f"{varY} ~ {varZ}", data=data).fit()

X = model_SecondFlrSF.resid
y = model_SalePrice.resid
model_res = sm.OLS(y, X).fit()

fig = go.Figure()

fig.add_trace(go.Scatter(
        x=model_SecondFlrSF.resid,
        y=model_SalePrice.resid,
        mode='markers',
        line=dict(color='#1f77b4'),
    ))

fig.update_layout(
        height=500, width=1200,
        title=dict(text=f"Regressing residuals. Partial Correlation ({varX},{varY}), adjusting for {varZ} is R^2 = {np.round(model_res.rsquared,3)}", x=0.5),
        xaxis_title=f'Residuals {varX} ~ {varZ}',
        yaxis_title=f'Residuals {varY} ~ {varZ}',
        template='plotly_white',
        legend=dict(orientation='h', x=.5, y=1, xanchor='center', yanchor='bottom')
    )

fig.show()

In [7]:
r, p = pearsonr(model_SecondFlrSF.resid, model_SalePrice.resid)
print(f"The partial correlation between {varX}, {varY}, adjusted for {varZ} is {r:.4f} with p-value = {p:.5f}")

The partial correlation between SecondFlrSF, LogSalePrice, adjusted for TotRmsAbvGrd is -0.0478 with p-value = 0.15550


The partial correlation, is positive with a significant p-value, meaning that TotRmsAbvGrd is correlated with both SecondFlrSF AND LogSalePrice, so increasing SecondFlrSF will not increase LogSalePrice, and the company should not try to maximize second floor area. Their decision does not make sense. 